In [1]:
# Cell 1 — LOAD DATA + GRAPH (robust paths)

import os
from pathlib import Path
import pandas as pd
import osmnx as ox
import joblib

# --- Find repo root (folder that contains /data and /model) ---
HERE = Path.cwd()

def find_repo_root(start: Path) -> Path:
    for p in [start] + list(start.parents):
        if (p / "data").exists() and (p / "model").exists():
            return p
    return start  # fallback

REPO = find_repo_root(HERE)
os.chdir(REPO)  # force correct working directory

print("📌 Loading project files...")
print("Working dir:", Path.cwd())

# --- Load files ---
data_path = REPO / "data" / "atlanta_stops.csv"
model_path = REPO / "model" / "travel_time_model.pkl"
baseline_path = REPO / "model" / "route_baseline.pkl"
ai_path = REPO / "model" / "route_ai.pkl"

print("Looking for:", data_path)

df = pd.read_csv(data_path)

model = joblib.load(model_path)
route_baseline = joblib.load(baseline_path)
route_ai = joblib.load(ai_path)

print("Stops loaded:", len(df))
print("Routes loaded:", len(route_baseline), "(baseline) |", len(route_ai), "(AI)")

# --- Load road graph ---
bbox = {"north": 33.92, "south": 33.55, "east": -84.25, "west": -84.55}
G = ox.graph_from_bbox(
    bbox["north"], bbox["south"], bbox["east"], bbox["west"], network_type="drive"
)

print("Graph loaded with", len(G.nodes), "nodes")


📌 Loading project files...
Working dir: /Users/akhilpartheeban/Documents/ai-route-engine
Looking for: /Users/akhilpartheeban/Documents/ai-route-engine/data/atlanta_stops.csv
Stops loaded: 100
Routes loaded: 99 (baseline) | 99 (AI)


/var/folders/z9/qlkdzbln0z172qmjwkffnt8h0000gn/T/ipykernel_72444/908132390.py:43: FutureWarning: The `north`, `south`, `east`, and `west` parameters are deprecated and will be removed in the v2.0.0 release. Use the `bbox` parameter instead. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  G = ox.graph_from_bbox(


Graph loaded with 39900 nodes


In [2]:
# Cell 2 — Build full road paths for both routes
def build_path(route, G):
    full = []
    dist = 0
    for i in range(len(route) - 1):
        s = nx.shortest_path(G, route[i], route[i+1], weight="length")
        d = nx.shortest_path_length(G, route[i], route[i+1], weight="length")
        dist += d
        if full:
            s = s[1:]
        full.extend(s)
    return full, dist

baseline_path, baseline_full_dist = build_path(route_baseline, G)
ai_path, ai_full_dist = build_path(route_ai, G)

print("Baseline distance (m):", baseline_full_dist)
print("AI distance (m):", ai_full_dist)


NameError: name 'nx' is not defined

In [ ]:
# Cell 3 — Folium interactive map (recommended)
import folium
import osmnx as ox

# Create map centered on Atlanta
m = folium.Map(location=[33.75, -84.39], zoom_start=11, tiles="cartodbpositron")

# -------- Feature Groups (THIS IS THE KEY PART) --------
fg_baseline = folium.FeatureGroup(name="Baseline Route")
fg_ai = folium.FeatureGroup(name="AI Route")
fg_stops = folium.FeatureGroup(name="Stops")

# -------- Draw baseline route (RED) --------
for i in range(len(route_baseline) - 1):
    try:
        path = ox.shortest_path(G, route_baseline[i], route_baseline[i+1], weight="length")
        coords = [(G.nodes[n]['y'], G.nodes[n]['x']) for n in path]
        folium.PolyLine(coords, color="red", weight=3).add_to(fg_baseline)
    except:
        pass

# -------- Draw AI route (BLUE) --------
for i in range(len(route_ai) - 1):
    try:
        path = ox.shortest_path(G, route_ai[i], route_ai[i+1], weight="length")
        coords = [(G.nodes[n]['y'], G.nodes[n]['x']) for n in path]
        folium.PolyLine(coords, color="blue", weight=3).add_to(fg_ai)
    except:
        pass

# -------- Draw stops (BLACK DOTS) --------
for node in route_ai:
    folium.CircleMarker(
        location=(G.nodes[node]['y'], G.nodes[node]['x']),
        radius=4,
        color="black",
        fill=True,
        fill_opacity=0.9
    ).add_to(fg_stops)

# -------- Add groups to map --------
fg_baseline.add_to(m)
fg_ai.add_to(m)
fg_stops.add_to(m)

# -------- THIS LINE CREATES THE TOGGLE --------
folium.LayerControl(collapsed=False).add_to(m)

# Save map
os.makedirs("visuals", exist_ok=True)
m.save("visuals/routes_map.html")

print("✅ Map with toggles saved → visuals/routes_map.html")


In [ ]:
# Cell 4 — Compute KPI metrics
BASE_SPEED = 10  # m/s

baseline_km = baseline_full_dist / 1000
ai_km = ai_full_dist / 1000

baseline_hr = baseline_full_dist / BASE_SPEED / 3600
ai_hr = ai_full_dist / BASE_SPEED / 3600

dist_change = (ai_km - baseline_km) / baseline_km * 100
time_change = (ai_hr - baseline_hr) / baseline_hr * 100

print("Baseline distance (km):", baseline_km)
print("AI distance (km):", ai_km)
print("Distance change (%):", dist_change)

print("\nBaseline time (hr):", baseline_hr)
print("AI time (hr):", ai_hr)
print("Time change (%):", time_change)


In [ ]:
# Cell 5 — KPI Dashboard
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2, figsize=(12, 5))

# Distance bar chart
ax[0].bar(["Baseline", "AI"], [baseline_km, ai_km], color=["gray", "blue"])
ax[0].set_title("Total Route Distance (km)")

# Time chart
ax[1].bar(["Baseline", "AI"], [baseline_hr, ai_hr], color=["gray", "blue"])
ax[1].set_title("Total Travel Time (hr)")

plt.suptitle("Performance — AI vs Baseline Routing")
plt.show()


In [ ]:
# FINAL KPI — Estimated Emissions Impact

# Assumptions
CO2_PER_KM = 0.251  # kg CO2 per km (EPA avg conversion)
BASE_SPEED = 10  # m/s

baseline_km = baseline_full_dist / 1000
ai_km = ai_full_dist / 1000

baseline_co2 = baseline_km * CO2_PER_KM
ai_co2 = ai_km * CO2_PER_KM

co2_improvement = (baseline_co2 - ai_co2) / baseline_co2 * 100

print("Estimated CO₂ emissions (kg):")
print("Baseline:", round(baseline_co2, 2))
print("AI:", round(ai_co2, 2))
print(f"Emissions reduction: {co2_improvement:.2f}%")
